# Семинар 1. Базовая работа с прочтениями. Домашнее задание

Загрузите прочтения всех экспериментов из статьи Evaluation of two main RNA-seq approaches for gene quantification in clinical RNA sequencing: polyA+ selection versus rRNA depletion, в которой сравнивают два протокола подготовки библиотек для bulk RNA-Seq: при помощи деплеции рРНК (=total) или при помощи селекции polyA-транскриптов (=polyA+).

При выполнении задания используйте Nextflow pipeline.

### SRA Toolkit

Для того, чтобы загрузить SRA-эксперимент и преобразовать его в .fastq-прочтения (мы сделаем это одновременно одной командой, однако можно выполнить их независимо), необходимо выполнить следующие команды. Сначала мы загрузим SRA Toolkit с сайта NCBI:

In [ ]:
!wget https://ftp-trace.ncbi.nlm.nih.gov/sra/sdk/3.0.0/sratoolkit.3.0.0-ubuntu64.tar.gz
!tar -xvzf sratoolkit.3.0.0-ubuntu64.tar.gz

--2022-09-12 11:20:59--  https://ftp-trace.ncbi.nlm.nih.gov/sra/sdk/3.0.0/sratoolkit.3.0.0-ubuntu64.tar.gz
Resolving ftp-trace.ncbi.nlm.nih.gov (ftp-trace.ncbi.nlm.nih.gov)... 130.14.250.10, 130.14.250.11, 2607:f220:41e:250::13, ...
Connecting to ftp-trace.ncbi.nlm.nih.gov (ftp-trace.ncbi.nlm.nih.gov)|130.14.250.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85777118 (82M) [application/x-gzip]
Saving to: ‘sratoolkit.3.0.0-ubuntu64.tar.gz’

sratoolkit.3.0.0-ub 100%[===================>]  81.80M  11.7MB/s    in 7.3s    

2022-09-12 11:21:07 (11.2 MB/s) - ‘sratoolkit.3.0.0-ubuntu64.tar.gz’ saved [85777118/85777118]

sratoolkit.3.0.0-ubuntu64/
sratoolkit.3.0.0-ubuntu64/bin/
sratoolkit.3.0.0-ubuntu64/bin/abi-dump
sratoolkit.3.0.0-ubuntu64/bin/abi-dump.3
sratoolkit.3.0.0-ubuntu64/bin/abi-load
sratoolkit.3.0.0-ubuntu64/bin/abi-load.3
sratoolkit.3.0.0-ubuntu64/bin/align-info
sratoolkit.3.0.0-ubuntu64/bin/align-info.3
sratoolkit.3.0.0-ubuntu64/bin/bam-load
sratoolk

Нам необходимо загрузить прочтения при помощи программы fasterq-dump из SRA Toolkit. Внутри Google Colab настроить SRA Toolkit правильно крайне сложно, поэтому необходимо будет выполнить некоторую странную процедуру: специально вызвать Segmentation fault у инструмента vdb-config.

In [ ]:
!sratoolkit.3.0.0-ubuntu64/bin/vdb-config -i

2022-09-12T11:21:46 vdb-config.3.0.0 fatal: SIGNAL - Segmentation fault 


Теперь мы можем спокойно запустить fastq-dump и разархивировать наши прочтения

# Загружаем NextFlow, настраиваем

NextFlow — это специальный язык (groovy), который позволяет вам запускать целые пайплайны из нескольких последовательно выполняемых программ, следя за тем, чтобы каждая стадия выполнялась строго одна за одной.

In [ ]:
!curl -fsSL https://get.nextflow.io | bash

CAPSULE: Downloading dependency org.multiverse:multiverse-core:jar:0.7.0
CAPSULE: Downloading dependency ch.qos.logback:logback-classic:jar:1.2.9
CAPSULE: Downloading dependency org.checkerframework:checker-compat-qual:jar:2.0.0
CAPSULE: Downloading dependency org.codehaus.mojo:animal-sniffer-annotations:jar:1.14
CAPSULE: Downloading dependency com.google.errorprone:error_prone_annotations:jar:2.1.3
CAPSULE: Downloading dependency org.codehaus.groovy:groovy-templates:jar:3.0.10
CAPSULE: Downloading dependency com.google.guava:guava:jar:24.1.1-jre
CAPSULE: Downloading dependency org.codehaus.jsr166-mirror:jsr166y:jar:1.7.0
CAPSULE: Downloading dependency io.nextflow:nf-commons:jar:22.04.5
CAPSULE: Downloading dependency org.slf4j:jcl-over-slf4j:jar:1.7.32
CAPSULE: Downloading dependency com.beust:jcommander:jar:1.35
CAPSULE: Downloading dependency org.slf4j:jul-to-slf4j:jar:1.7.32
CAPSULE: Downloading dependency com.google.j2objc:j2objc-annotations:jar:1.1
CAPSULE: Downloading dependenc

# Загружаем прочтения


Создала файл pr_1_hw.nf со следующим содержанием:

Для того, чтобы обкспечить параллельный запуск сразу нескольких процессов одним и тем же пайплайном, необходимо распарсить аргумент, который мы подаём в переменную <code>SRA</code> как список, после чего запустить наш немного модифицированный пайплайн:
```
params.results_dir = "results/"
params.SRA = "SRR000000"

log.info ""
log.info "  Q U A L I T Y   C O N T R O L  "
log.info "================================="
log.info "SRA number         : ${params.SRA}"
log.info "Results location   : ${params.results_dir}"

process DownloadFastQ {
  publishDir "${params.results_dir}"

  output:
    path "reads/*"

  script:
    """
    /content/sratoolkit.3.0.0-ubuntu64/bin/fasterq-dump ${params.SRA} -O reads/
    """  
}

workflow {
  DownloadFastQ()  
}
```


В статье прочтения под номером SRR6410603

In [ ]:
!./nextflow run /content/drive/MyDrive/pr_1_hw.nf --SRA SRR6410603

N E X T F L O W  ~  version 22.04.5
Launching `/content/drive/MyDrive/pr_1_hw.nf` [clever_mirzakhani] DSL2 - revision: fa5df5b637

  Q U A L I T Y   C O N T R O L  
SRA number         : SRR6410603
Results location   : results/
[-        ] process > DownloadFastQ -

[-        ] process > DownloadFastQ [  0%] 0 of 1

executor >  local (1)
[9e/52b56e] process > DownloadFastQ [  0%] 0 of 1

executor >  local (1)
[9e/52b56e] process > DownloadFastQ [100%] 1 of 1 ✔

executor >  local (1)
[9e/52b56e] process > DownloadFastQ [100%] 1 of 1 ✔
Completed at: 12-Sep-2022 12:11:51
Duration    : 17m 57s
CPU hours   : 0.3
Succeeded   : 1


